In [52]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import logging

logging.basicConfig(
    format="%(asctime)s ; %(levelname)s ; %(message)s",
    level=logging.DEBUG
)
logging.getLogger("scapy").setLevel(logging.CRITICAL)
log = logging.getLogger("adAPT")

from pathlib import Path
import tensorflow as tf
import pandas as pd
import numpy as np
from collections import Counter


In [54]:
b_parquet = "./data/benign_features.parquet"
m_parquet = "./data/malicious_features.parquet"

b_df = pd.read_parquet(b_parquet)
m_df = pd.read_parquet(m_parquet)

In [55]:
b_df

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,parsed
0,IP,Unknown,192.168.110.10,80.237.133.136,1152,80,52,b'E\x00\x004\x10M@\x00\x80\x06\x00\x00\xc0\xa8...,52,b'E\x00\x004\x10M@\x00\x80\x06\x00\x00\xc0\xa8...,None
1,IP,Unknown,80.237.133.136,192.168.110.10,80,1152,52,b'E\x00\x004\x00\x00@\x006\x06?\x9cP\xed\x85\x...,52,b'E\x00\x004\x00\x00@\x006\x06?\x9cP\xed\x85\x...,None
2,IP,Unknown,192.168.110.10,80.237.133.136,1152,80,40,b'E\x00\x00(\x10N@\x00\x80\x06\x00\x00\xc0\xa8...,40,b'E\x00\x00(\x10N@\x00\x80\x06\x00\x00\xc0\xa8...,None
3,IP,HTTPRequest,192.168.110.10,80.237.133.136,1152,80,351,b'E\x00\x01_\x10O@\x00\x80\x06\x00\x00\xc0\xa8...,351,b'E\x00\x01_\x10O@\x00\x80\x06\x00\x00\xc0\xa8...,"{'method': 'GET', 'path': '/', 'host': 'ip.web..."
4,IP,Unknown,80.237.133.136,192.168.110.10,80,1152,40,b'E\x00\x00(:\xad@\x006\x06\x04\xfbP\xed\x85\x...,40,b'E\x00\x00(:\xad@\x006\x06\x04\xfbP\xed\x85\x...,None
...,...,...,...,...,...,...,...,...,...,...,...
29954,UDP,Unknown,194.247.5.27,194.247.5.1,1812,10048,40,"b""\x07\x14'@\x00(\x90D""",40,b'E\x00\x00<\xbc\x15\x00\x00@\x11.\x91\xc2\xf7...,None
29955,UDP,Unknown,194.247.5.1,194.247.5.27,3860,1812,112,"b'\x0f\x14\x07\x14\x00p\xf9""'",112,b'E\x00\x00\x84\x9f\xd0\x00\x00@\x11J\x8e\xc2\...,None
29956,UDP,Unknown,194.247.5.27,194.247.5.1,1812,3860,104,b'\x07\x14\x0f\x14\x00h\x90\x84',104,b'E\x00\x00|\xc5\xb8\x00\x00@\x11$\xae\xc2\xf7...,None
29957,UDP,Unknown,194.247.5.1,194.247.5.27,3373,1812,120,b'\r-\x07\x14\x00x\xc0\x82',120,b'E\x00\x00\x8c\x9f\xd1\x00\x00@\x11J\x85\xc2\...,None


In [56]:
b_df[~b_df.parsed.isna()]

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,parsed
3,IP,HTTPRequest,192.168.110.10,80.237.133.136,1152,80,351,b'E\x00\x01_\x10O@\x00\x80\x06\x00\x00\xc0\xa8...,351,b'E\x00\x01_\x10O@\x00\x80\x06\x00\x00\xc0\xa8...,"{'method': 'GET', 'path': '/', 'host': 'ip.web..."
5,IP,HTTPResponse,80.237.133.136,192.168.110.10,80,1152,1500,b'E\x00\x05\xdc:\xae@\x006\x06\xffEP\xed\x85\x...,1500,b'E\x00\x05\xdc:\xae@\x006\x06\xffEP\xed\x85\x...,"{'connection': 'Keep-Alive', 'content_length':..."
6,IP,HTTPResponse,80.237.133.136,192.168.110.10,80,1152,83,b'E\x00\x00S:\xaf@\x006\x06\x04\xceP\xed\x85\x...,83,b'E\x00\x00S:\xaf@\x006\x06\x04\xceP\xed\x85\x...,{'error': 'could not parse packet data'}
16,IP,HTTPRequest,192.168.110.10,212.144.254.123,1154,3128,374,b'E\x00\x01v\x10Y@\x00\x80\x06\x00\x00\xc0\xa8...,374,b'E\x00\x01v\x10Y@\x00\x80\x06\x00\x00\xc0\xa8...,"{'method': 'GET', 'path': 'http://ip.webernetz..."
17,IP,HTTPRequest,192.168.110.10,212.144.254.123,1154,3128,374,b'E\x00\x01v\x10Z@\x00\x80\x06\x00\x00\xc0\xa8...,374,b'E\x00\x01v\x10Z@\x00\x80\x06\x00\x00\xc0\xa8...,"{'method': 'GET', 'path': 'http://ip.webernetz..."
...,...,...,...,...,...,...,...,...,...,...,...
29940,UDP,DNSQueryRequest,192.168.110.35,85.215.94.29,50821,53,60,b'\xc6\x85\x005\x00<\xe3\r',60,b'E\x00\x00Pm\xb9\x00\x00\x80\x11\x00\x00\xc0\...,{'error': 'could not parse packet data'}
29941,UDP,DNSQueryRequest,192.168.110.35,85.215.94.29,55577,53,60,b'\xd9\x19\x005\x00<\xe3\r',60,b'E\x00\x00Pm\xba\x00\x00\x80\x11\x00\x00\xc0\...,{'error': 'could not parse packet data'}
29942,UDP,DNSQueryRequest,192.168.110.35,85.215.94.29,55851,53,60,b'\xda+\x005\x00<\xe3\r',60,b'E\x00\x00Pm\xbb\x00\x00\x80\x11\x00\x00\xc0\...,{'error': 'could not parse packet data'}
29943,UDP,DNSQueryRequest,192.168.110.35,85.215.94.29,55889,53,60,b'\xdaQ\x005\x00<\xe3\r',60,b'E\x00\x00Pm\xbc\x00\x00\x80\x11\x00\x00\xc0\...,{'error': 'could not parse packet data'}


In [57]:
b_df[b_df.protocol == "UDP"]

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,parsed
31,UDP,Unknown,127.0.0.1,127.0.0.1,34751,5246,228,b'\x87\xbf\x14~\x00\xe4\x00\x00',228,b'E\x00\x00\xf8\x00\x00@\x00@\x11;\xf3\x7f\x00...,None
32,UDP,Unknown,127.0.0.1,127.0.0.1,5246,34751,133,b'\x14~\x87\xbf\x00\x85\x00\x00',133,b'E\x00\x00\x99\x00\x00@\x00@\x11<R\x7f\x00\x0...,None
33,UDP,Unknown,127.0.0.1,127.0.0.1,34751,5246,167,b'\x87\xbf\x14~\x00\xa7\x00\x00',167,b'E\x00\x00\xbb\x00\x00@\x00@\x11<0\x7f\x00\x0...,None
34,UDP,Unknown,127.0.0.1,127.0.0.1,5246,34751,169,b'\x14~\x87\xbf\x00\xa9\x00\x00',169,b'E\x00\x00\xbd\x00\x00@\x00@\x11<.\x7f\x00\x0...,None
35,UDP,Unknown,127.0.0.1,127.0.0.1,34751,5246,39,"b""\x87\xbf\x14~\x00'\x00\x00""",39,b'E\x00\x00;\x00\x00@\x00@\x11<\xb0\x7f\x00\x0...,None
...,...,...,...,...,...,...,...,...,...,...,...
29954,UDP,Unknown,194.247.5.27,194.247.5.1,1812,10048,40,"b""\x07\x14'@\x00(\x90D""",40,b'E\x00\x00<\xbc\x15\x00\x00@\x11.\x91\xc2\xf7...,None
29955,UDP,Unknown,194.247.5.1,194.247.5.27,3860,1812,112,"b'\x0f\x14\x07\x14\x00p\xf9""'",112,b'E\x00\x00\x84\x9f\xd0\x00\x00@\x11J\x8e\xc2\...,None
29956,UDP,Unknown,194.247.5.27,194.247.5.1,1812,3860,104,b'\x07\x14\x0f\x14\x00h\x90\x84',104,b'E\x00\x00|\xc5\xb8\x00\x00@\x11$\xae\xc2\xf7...,None
29957,UDP,Unknown,194.247.5.1,194.247.5.27,3373,1812,120,b'\r-\x07\x14\x00x\xc0\x82',120,b'E\x00\x00\x8c\x9f\xd1\x00\x00@\x11J\x85\xc2\...,None


In [58]:
ignore = [53, 67, 68, 5246, 5247, 1812, 520, 521, 1985, 123, 514, 1967]
b_df[~(b_df.dest_port.isin(ignore)) & ~(b_df.source_port.isin(ignore)) & (b_df.protocol == "UDP")]

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,parsed
302,UDP,Unknown,192.168.121.2,192.168.121.254,64199,65535,40,b'\xfa\xc7\xff\xff\x00(\xce\x1b',40,b'E\x00\x00<\x00\x01\x00\x00\xff\x11G^\xc0\xa8...,None
303,UDP,Unknown,192.168.121.2,192.168.121.253,64091,65534,40,b'\xfa[\xff\xfe\x00(\xce\x88',40,b'E\x00\x00<\x00\x01\x00\x00\xff\x11G_\xc0\xa8...,None
304,UDP,Unknown,192.168.121.254,192.168.121.2,65535,64199,40,b'\xff\xff\xfa\xc7\x00(j\x1b',40,b'E\x00\x00<\x00\x00\x00\x00\xff\x11G_\xc0\xa8...,None
305,UDP,Unknown,192.168.121.253,192.168.121.2,65534,64091,40,b'\xff\xfe\xfa[\x00(l4',40,b'E\x00\x00<\x00\x00\x00\x00\xff\x11G`\xc0\xa8...,None
306,UDP,Unknown,192.168.121.2,192.168.121.254,64199,65535,40,b'\xfa\xc7\xff\xff\x00(\xce\x05',40,b'E\x00\x00<\x00\x02\x00\x00\xff\x11G]\xc0\xa8...,None
...,...,...,...,...,...,...,...,...,...,...,...
29744,UDP,Unknown,192.168.3.85,5.35.226.136,60373,33474,40,b'\xeb\xd5\x82\xc2\x00(\xab\xe2',40,b'E\x00\x00<=A\x00\x00\x0e\x11\xc3\xc7\xc0\xa8...,None
29745,UDP,Unknown,192.168.3.85,5.35.226.136,53756,33475,40,b'\xd1\xfc\x82\xc3\x00(\xab\xe2',40,b'E\x00\x00<=B\x00\x00\x0e\x11\xc3\xc6\xc0\xa8...,None
29746,UDP,Unknown,192.168.3.85,5.35.226.136,52401,33476,40,b'\xcc\xb1\x82\xc4\x00(\xab\xe2',40,b'E\x00\x00<=C\x00\x00\x0f\x11\xc2\xc5\xc0\xa8...,None
29747,UDP,Unknown,192.168.3.85,5.35.226.136,46398,33477,40,b'\xb5>\x82\xc5\x00(\xab\xe2',40,b'E\x00\x00<=D\x00\x00\x0f\x11\xc2\xc4\xc0\xa8...,None
